In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
from selenium import webdriver

일단 무작정 import하고 쓰기 시작했는데 chrome devtools에서는 데이터가 모두 보이는데 이렇게만 읽어오면 내가 원하는 데이터는 보이지 않는다.

In [111]:
res = requests.post('https://kbl.or.kr/game/archive-team')
print(res.content)

b'<!doctype html><html lang="ko"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no"/><title>KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80</title><meta name="author" content="KBL"/><meta name="description" content="KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80"/><meta name="referrer" content="no-referrer"/><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="Expires" content="-1"/><meta http-equiv="Pragma" content="no-cache"><meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate"/><link rel="icon" href="/imgs/favicon/kbl/favicon.svg"><link rel="apple-touch-icon" type="image/png" id="apple-icon-60x60" sizes="60x60" href="/imgs/favicon/kbl/apple-icon-60x60.png"><link rel="apple-touch-icon" type="image/png" id="apple-icon-87x87" sizes="87x87" href="/imgs/favicon/kbl/apple-icon-87x87.png"><link r

In [92]:
soup = BeautifulSoup(res.content, 'html.parser')

In [97]:
soup.find_all('div', class_='archive-team-table01-wrap')

[]

검색해 본 결과, 위의 코드는 정적 페이지의 데이터를 읽어오는 것이고 동적 페이지는 webdriver를 사용해서 진행해야 내가 원하는 데이터를 가져올 수 있는 것이다.

- 그런데 아무리 사용해도 나오지 계속 같은 결과만을 가져왔다.
- 그러나~!!, ```driver.page_source```  하니까 나왔다 드디어!!!ㅜㅜ 감격스러움

- class를 더 쪼개고 해서 더 분할해서 데이터를 저장하는 것이 좋을 것 같다~

In [105]:
driver = webdriver.Chrome('chromedriver_win32/')
# driver.implicitly_wait(3) # 있으나 없으나
driver.get('https://kbl.or.kr/game/archive-team')

C:\Users\Kwon\AppData\Local\Temp\ipykernel_436728\4060881888.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_win32/')


In [108]:
from selenium.webdriver.common.by import By

In [112]:
driver.find_elements(By.CLASS_NAME, 'con-box')

[<selenium.webdriver.remote.webelement.WebElement (session="cd3c949803d492622ec3a130bb34e157", element="6bb2d47d-8d65-472c-b19f-9ed0496bd676")>]

In [114]:
html = driver.page_source # page_source를 가져와야 하는 것이었다ㅠㅠ 감격스러움 진짜
soup = BeautifulSoup(html, 'html.parser')

In [138]:
f = open('temp.html', 'w')
f.write(s)
f.close()

TypeError: write() argument must be str, not method

In [167]:
f.close()

In [127]:
type(soup)

bs4.BeautifulSoup

In [151]:
ths = soup.select('th') # 테이블 헤더 제목

In [152]:
for h in ths:
    print(h.text.strip())

순위
팀
G
PTS
2P
2PA
2P%
3P
3PA
3P%
FG
FGA
FG%
FT
FTA
FT%
PP
PPA
PP%
REBOUND
AST
TO
STL
BS
PF
OFF
DEF
TOT


In [155]:
noes = soup.select('td.name') # 순위 순서로 팀 이름

In [156]:
for no in noes:
    print(no.text.strip())

서울 SK
안양 KGC
울산 현대모비스
고양 캐롯
원주 DB
전주 KCC
대구 한국가스공사
창원 LG
수원 KT
서울 삼성


In [160]:
games = soup.select('td.bd-gray') # 각 팀 별 경기 횟수 마지막은 전체 경기 수

In [158]:
for game in games:
    print(game.text.strip())

23
23
22
22
22
23
22
22
22
23
224


In [161]:
records = soup.select('td.on')

In [162]:
for rec in records:
    print(rec.text.strip())

84.7
83.3
83.1
82.8
81.8
80.8
80.5
78.8
76.2
74.4


In [163]:
records = soup.select('td')

In [164]:
for r in records:
    print(r.text.strip())

1
서울 SK
23
2
안양 KGC
23
3
울산 현대모비스
22
4
고양 캐롯
22
5
원주 DB
22
6
전주 KCC
23
7
대구 한국가스공사
22
8
창원 LG
22
9
수원 KT
22
10
서울 삼성
23
합계
224
84.7
25.6
48.7
52.6
6.8
19.9
34.1
32.4
68.5
47.3
13.2
18.4
71.5
21.1
36.9
57.2
10.3
23.9
34.2
18.3
9.0
7.5
2.7
18.9
83.3
21.0
39.8
52.8
9.9
28.4
34.9
31.0
68.3
45.4
11.4
15.7
73.1
16.8
28.9
58.2
10.4
26.3
36.8
18.3
10.1
5.8
3.0
18.5
83.1
25.2
47.5
53.0
7.0
20.9
33.3
32.1
68.4
47.0
11.9
17.6
67.7
20.1
34.6
58.1
11.5
25.6
37.1
20.3
10.6
6.9
2.0
20.0
82.8
18.0
34.5
52.2
12.0
32.8
36.5
30.0
67.3
44.6
10.8
14.3
75.6
13.5
23.1
58.7
8.6
22.9
31.5
16.5
9.3
7.5
2.5
17.5
81.8
25.5
49.1
51.9
6.9
20.3
34.1
32.4
69.4
46.7
10.2
13.7
74.2
19.4
33.8
57.5
11.0
24.4
35.4
16.9
11.5
6.5
2.2
19.8
80.8
22.5
45.0
50.0
7.6
21.0
36.1
30.0
65.9
45.6
13.2
16.7
79.1
17.5
31.0
56.5
10.6
24.3
34.9
18.1
9.7
5.6
1.7
17.3
80.5
23.8
43.1
55.1
6.8
21.0
32.4
30.6
64.2
47.7
12.5
16.8
74.1
17.8
30.1
59.1
9.2
23.3
32.5
17.1
10.9
6.8
2.5
19.5
78.8
23.6
46.2
51.1
6.7
20.5
32.7
30.3
66.7
45.4
11.4
16.9

선수 기록실

In [165]:
driver = webdriver.Chrome('chromedriver_win32/')
driver.get('https://kbl.or.kr/game/archive-player')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

C:\Users\Kwon\AppData\Local\Temp\ipykernel_436728\389803932.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_win32/')


In [166]:
records = soup.select('td')

for r in records:
    print(r.text.strip())

1
레스터 프로스퍼
수원 KT
2
자밀 워니
서울 SK
3
오마리 스펠맨
안양 KGC
4
전성현
고양 캐롯
5
게이지 프림
울산 현대모비스
6
두경민
원주 DB
7
허웅
전주 KCC
7
드완 에르난데스
원주 DB
9
이대성
대구 한국가스공사
10
라건아
전주 KCC
11
디드릭 로슨
고양 캐롯
12
이정현
고양 캐롯
13
아셈 마레이
창원 LG
14
김선형
서울 SK
15
최준용
서울 SK
16
변준형
안양 KGC
17
이선 알바노
원주 DB
18
마커스 데릭슨
서울 삼성
19
이제이 아노시케
수원 KT
20
안정욱
고양 캐롯
21
론제이 아바리엔토스
울산 현대모비스
22
머피 할로웨이
대구 한국가스공사
23
오세근
안양 KGC
23
이정현
서울 삼성
23
하윤기
수원 KT
26
이우석
울산 현대모비스
27
저스틴 녹스
울산 현대모비스
28
양홍석
수원 KT
29
정성우
수원 KT
30
이재도
창원 LG
31
이매뉴얼 테리
서울 삼성
32
이승현
전주 KCC
33
허일영
서울 SK
33
데이비드 사이먼
고양 캐롯
35
정효근
대구 한국가스공사
36
이관희
창원 LG
37
렌즈 아반도
안양 KGC
38
론데 홀리스 제퍼슨
전주 KCC
38
정창영
전주 KCC
38
이대헌
대구 한국가스공사
41
이원석
서울 삼성
42
레나드 프리먼
원주 DB
43
단테 커닝햄
창원 LG
44
강상재
원주 DB
44
최성원
서울 SK
46
김시래
서울 삼성
47
함지훈
울산 현대모비스
48
서명진
울산 현대모비스
49
이호현
서울 삼성
50
유슈 은도예
대구 한국가스공사
51
문성곤
안양 KGC
51
샘조세프 벨란겔
대구 한국가스공사
53
윤원상
창원 LG
53
김종규
원주 DB
55
최승욱
원주 DB
56
김준일
창원 LG
56
배병준
안양 KGC
58
장재석
울산 현대모비스
58
한희원
수원 KT
58
조나단 알렛지
서울 삼성
61
이근휘
전주 KCC
61
리온 윌리엄스
서울 SK
61
최현민
고양 캐롯
64
박지훈
안양 KGC
64
오재현
서울 SK
66
이두원
수원 KT
6

In [179]:
records = soup.select('.top-scroll-table.archive-player-table01 td')

for r in records:
    print(r.text.strip())

31.0
11.0
21.0
52.4
5.0
6.0
83.3
7.0
13.0
53.8
2.0
2.0
4.0
1.0
3.0
1.0
0.0
2.0
23.1
10.0
19.3
51.9
2.7
4.3
62.0
8.9
15.7
56.9
2.7
6.9
9.6
2.3
1.7
1.0
1.3
1.6
19.4
7.1
16.2
43.7
2.3
3.3
72.0
3.3
5.9
56.6
2.3
7.3
9.5
2.5
1.8
0.8
1.0
2.3
18.9
6.0
14.2
42.6
3.0
3.4
89.2
1.4
2.7
50.0
0.3
1.5
1.9
3.0
2.5
1.3
0.0
1.3
17.8
7.1
12.1
58.6
3.6
5.0
72.5
6.5
10.5
61.6
4.0
6.7
10.7
2.5
1.8
1.6
0.7
3.1
16.8
5.7
12.8
44.2
2.6
2.8
94.0
1.7
3.1
54.5
0.5
1.9
2.4
2.9
1.8
0.9
0.1
2.6
16.6
5.3
11.9
44.9
3.5
4.0
87.0
2.4
4.3
55.6
0.2
2.1
2.3
5.0
2.2
1.0
0.0
1.9
16.6
6.9
13.7
50.2
2.6
4.0
65.6
5.8
10.6
54.1
1.8
5.4
7.2
1.6
2.3
0.2
0.7
2.1
16.5
6.1
13.7
44.4
3.2
3.8
85.5
3.5
6.1
58.2
0.3
2.2
2.5
3.9
2.2
1.5
0.2
2.1
16.4
6.6
12.1
54.5
2.6
3.2
80.8
5.4
8.5
64.1
4.0
8.3
12.3
1.5
1.6
0.3
0.8
1.7
16.2
6.6
13.1
50.7
1.6
2.3
72.0
4.8
7.5
63.3
2.1
6.8
8.9
3.0
1.7
1.2
1.0
1.8
15.6
5.3
11.7
45.3
2.5
3.2
80.0
2.1
4.3
48.4
0.5
2.0
2.5
3.6
1.4
1.9
0.1
2.6
15.1
6.0
10.0
59.7
3.1
5.3
59.0
5.9
9.5
61.7
4.2
8.6
12.9
1.3
2.1
1.

- 기록실 링크 정리

```https://kbl.or./game/*```

1. 일정: schedule-list
2. 팀 순위: team-rank
3. 선수 순위: player-rank-top5
4. 팀 기록실: archive-team
5. 선수 기록실: archive-player
6. 팀 기록 비교: compare-team
7. 공수력 비교: compare-offence-defence